# **Data Collection Notebook**

## Objectives

- Fetch student exam scores dataset from Kaggle
- Save dataset
- Inspect the dataset

## Inputs
- Kaggle JSON authentication token
- Kaggle dataset file

## Outputs
- Dataset as a CSV file in the outputs/datasets/collection directory

## Additional comments
- The dataset is hosted on Kaggle, and the dataset is anonymised, so there are no privacy concerns to deal with

# Change working directory

Since this notebook exists in the jupyter_notebooks directory, we need to change the current working directory from the jupyter_notebooks directory to the workspace, so that any directories created in further codes cells are added in the correct place. 

We access the current directory with the OS packages' `getcwd()` method

In [1]:
import os
current_directory = os.getcwd()
current_directory


'/workspace/Exam-Scores-Analysis/jupyter_notebooks'

We now want to set the working directory as the parent of the current working directory, jupyter_notebooks

- The `os.path.dirname()` method gets the parent directory
- The `os.chir()` method defines the new current directory
- We do this to access all of the project's files and directories, rather than those in the jupyter_notebooks directory

In [3]:
os.chdir(os.path.dirname(current_directory))
print("You set a new current directory")

You set a new current directory


To make certain of things, we now use a code cell to confirm that we have set the current working directory properly

In [4]:
current_directory = os.getcwd()
current_directory

'/workspace/Exam-Scores-Analysis'

## Fetch data from Kaggle

We are now in a position to fetch the dataset from Kaggle

First, we need to install the Kaggle package

In [6]:
! pip install kaggle==1.5.12

You should consider upgrading via the '/home/gitpod/.pyenv/versions/3.8.14/bin/python3.8 -m pip install --upgrade pip' command.


We upload the kaggle.json authentication token to the workspace. Then we run the code cell below to ensure that the code cell is recognised

In [7]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

The dataset is located [here](https://www.kaggle.com/datasets/whenamancodes/students-performance-in-exams)

We can now import it

In [8]:
KaggleDatasetPath = "whenamancodes/students-performance-in-exams"
DestinationFolder = "inputs/datasets/raw"   
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

  0%|                                               | 0.00/8.67k [00:00<?, ?B/s]
100%|██████████████████████████████████████| 8.67k/8.67k [00:00<00:00, 11.0MB/s]


The dataset has now been imported. It exists as a zipped file in the inputs/datasets/raw directory. In the code cell below, we unzip it. Unlike in the walkthrough project, the raw dataset and the kaggle token will not be deleted, as the kaggle token will not be pushed to the public repository. Both have been retained in case the dataset needs to be re-imported.

In [9]:
! unzip {DestinationFolder}/*.zip -d {DestinationFolder}

Archive:  inputs/datasets/raw/students-performance-in-exams.zip
  inflating: inputs/datasets/raw/exams.csv  


We can now begin to inspect the dataset

## Load and inspect Kaggle dataset

We can assign the dataset to a Pandas dataframe using the `read_csv()` method, and display the first 5 rows using the `head()` method

In [1]:
import pandas as pd
df = pd.read_csv(f"inputs/datasets/raw/exams.csv")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'inputs/datasets/raw/exams.csv'

We can now see how much of dataset we have to work with

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race/ethnicity               1000 non-null   object
 2   parental level of education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test preparation course      1000 non-null   object
 5   math score                   1000 non-null   int64 
 6   reading score                1000 non-null   int64 
 7   writing score                1000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 62.6+ KB


The dataset has 1000 records in 8 columns. This should be an appropriate number of records to train a machine learning model. 

The dataset also has no missing data, so data imputation will not be necessary. The dataset also contains no columns that could contain unique information, so handling this such as by dropping columns is also unnecessary. 

The gender, race/ethnicity, parental level of education, lunch and test preparation course feature variables contain categorical data, so the object data-type is practical. There is no need to convert data-types.

However, to improve the dataset's ease of use, it would be useful to convert the column headers (gender, race/ethnicity, etc) to snake_case, and to simplify them as follows:
- `race/ethnicity` converted to `ethnicity`
- `parental level of education` converted to `parental_education`
- `lunch` converted to `lunch_program`
- `test preparation course` to `test_preparation_course`
- `math score` to `math_score`
- `reading score` to `reading_score`
- `writing score` to `writing_score`

We can do this with the `columns()` method

In [13]:
df.columns = ['gender', 'ethnicity', 'parental_education', 'lunch_program', 'test_preparation_course', 'math_score', 'reading_score', 'writing_score']
df.head()

,gender,ethnicity,parental_education,lunch_program,test_preparation_course,math_score,reading_score,writing_score
0,male,group A,high school,standard,completed,67,67,63
1,female,group D,some high school,free/reduced,none,40,59,55
2,male,group E,some college,free/reduced,none,59,60,50
3,male,group B,high school,standard,none,77,78,68
4,male,group E,associate's degree,standard,completed,78,73,68


It would also be useful to add 2 more columns to help us better understand the data:
- a column called `average_score` that averages the 3 score columns to provide an overall indication of academic performance
- a column called `literacy_score` that averages the `reading_score` and `writing_score` to provide an overall indication of literacy

In [18]:
df['average_score'] = df[['math_score', 'reading_score', 'writing_score']].mean(axis=1)
df.head()

,gender,ethnicity,parental_education,lunch_program,test_preparation_course,math_score,reading_score,writing_score,average_score
0,male,group A,high school,standard,completed,67,67,63,65.666667
1,female,group D,some high school,free/reduced,none,40,59,55,51.333333
2,male,group E,some college,free/reduced,none,59,60,50,56.333333
3,male,group B,high school,standard,none,77,78,68,74.333333
4,male,group E,associate's degree,standard,completed,78,73,68,73.000000


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   gender                   1000 non-null   object 
 1   ethnicity                1000 non-null   object 
 2   parental_education       1000 non-null   object 
 3   lunch_program            1000 non-null   object 
 4   test_preparation_course  1000 non-null   object 
 5   math_score               1000 non-null   int64  
 6   reading_score            1000 non-null   int64  
 7   writing_score            1000 non-null   int64  
 8   average_score            1000 non-null   float64
dtypes: float64(1), int64(3), object(5)
memory usage: 70.4+ KB


If we run the info method, we see that the average_score column is a float. Given that the test scores are integers, we can make the reasonable assumption that the exams do not allow for fractional marks. Therefore, we should convert the average_score column to an integer. The `astype()` method automatically rounds the value, and then saves the changes. 

In [32]:
df = df.astype({'average_score':'int'})
df.head()

,gender,ethnicity,parental_education,lunch_program,test_preparation_course,math_score,reading_score,writing_score,average_score
0,male,group A,high school,standard,completed,67,67,63,65
1,female,group D,some high school,free/reduced,none,40,59,55,51
2,male,group E,some college,free/reduced,none,59,60,50,56
3,male,group B,high school,standard,none,77,78,68,74
4,male,group E,associate's degree,standard,completed,78,73,68,73


If we check the data-type again, we see that average_score has been converted to a integer.

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   gender                   1000 non-null   object
 1   ethnicity                1000 non-null   object
 2   parental_education       1000 non-null   object
 3   lunch_program            1000 non-null   object
 4   test_preparation_course  1000 non-null   object
 5   math_score               1000 non-null   int64 
 6   reading_score            1000 non-null   int64 
 7   writing_score            1000 non-null   int64 
 8   average_score            1000 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 70.4+ KB


We can now add the `literacy_score` column

In [34]:
df['literacy_score'] = df[['reading_score', 'writing_score']].mean(axis=1)
df.head()

,gender,ethnicity,parental_education,lunch_program,test_preparation_course,math_score,reading_score,writing_score,average_score,literacy_score
0,male,group A,high school,standard,completed,67,67,63,65,65.0
1,female,group D,some high school,free/reduced,none,40,59,55,51,57.0
2,male,group E,some college,free/reduced,none,59,60,50,56,55.0
3,male,group B,high school,standard,none,77,78,68,74,73.0
4,male,group E,associate's degree,standard,completed,78,73,68,73,70.5


As before, we need to convert the data-types

In [35]:
df = df.astype({'literacy_score':'int'})
df.head()

,gender,ethnicity,parental_education,lunch_program,test_preparation_course,math_score,reading_score,writing_score,average_score,literacy_score
0,male,group A,high school,standard,completed,67,67,63,65,65
1,female,group D,some high school,free/reduced,none,40,59,55,51,57
2,male,group E,some college,free/reduced,none,59,60,50,56,55
3,male,group B,high school,standard,none,77,78,68,74,73
4,male,group E,associate's degree,standard,completed,78,73,68,73,70


And we confirm that the dataframe has been modified with the `info()` method

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   gender                   1000 non-null   object
 1   ethnicity                1000 non-null   object
 2   parental_education       1000 non-null   object
 3   lunch_program            1000 non-null   object
 4   test_preparation_course  1000 non-null   object
 5   math_score               1000 non-null   int64 
 6   reading_score            1000 non-null   int64 
 7   writing_score            1000 non-null   int64 
 8   average_score            1000 non-null   int64 
 9   literacy_score           1000 non-null   int64 
dtypes: int64(5), object(5)
memory usage: 78.2+ KB


Now that we have modified the dataframe appropriately, we can save it as a CSV file, and push the file to the repository. We will create an outputs directory

In [ ]:
import os
try:
  os.makedirs(name='outputs/datasets/collection') # create outputs/datasets/collection folder
except Exception as e:
  print(e)

df.to_csv(f"outputs/datasets/collection/student-exam-results.csv", index=False)